In [3]:
import pandas as pd

In [35]:
year = 2021
locate = "강남구"


def population(year, locate):
    df = pd.read_csv(r"./data/주민등록인구_20230710130833.csv", header=0, index_col="동별(2)")
    df.drop(columns="동별(1)", inplace=True)
    df.drop(index="동별(2)", inplace=True)
    df.drop(index="소계", inplace=True)
    return df[str(year)][str(locate)]


def parkinglot(year, locate):
    df = pd.read_csv(r"./data/주택가+주차장_20230710095648.csv", header=0, index_col="자치구별(2)")
    df.drop(columns='자치구별(1)',inplace=True)
    print(df[str(year)+'.5'][locate])
    df.head()

# def house_price(year,locate):
df=pd.read_csv(r'data/주택가격지수(매매)_20230710095430.csv',header=0)
df.loc[df['시점'] == year].loc[df['자치구별(2)']=='강남구']


,시점,자치구별(1),자치구별(2),종합,아파트
542,2021,서울시,강남구,105.1,105.8


In [9]:
import pandas as pd

# 예제 데이터프레임 생성
data = {'A': ['12345', '12346', '12347', '12348', '12349']}
df = pd.DataFrame(data)

# 컬럼 A의 마지막 문자가 5가 아닌 행 제거
df = df[df['A'].str[-1] == '5']

print(df)

       A
0  12345
